Notebook for using Kurve to build 
datasets for the user-engagement problem
from the Relational Deep Learning benchmark: https://relbench.stanford.edu/


In [ ]:
# Step 1: create an account at https://demo.kurve.ai

# Step 2: create a graph on the /usr/local/lake/rel-stack sample data with the following build parameters:
- edge count factor = 4
- use LLM for primary key = NO
- run inclusion / join checks = YES

# step 3: visit the metadata graph after it finished building
- click on <b>Actions</b> on the nav bar and select <b>Assign Parent Node</b>
- select the Users table and a depth limit of 4
- click on <b>Actions</b> again and now select <b>Compute Graph</b>
- Add the following parameters

name: user badge<br>
parent node: User<br>
depth limit: 4<br>
compute period in days: 3650<br>
cut date: 01/01/2020<br>
label period in days: 90<br>
label node: Post<br>
label field: Id<br>
label operation: bool<br>

# step 4: add the label generation SQL for the other targets
## Since the user engagement problem is to predict whether the user will make a Post, Vote, or Comment we need to ensure the other nodes have label generation logic.

- If there is not a Votes node directly below the User click on <b>Actions</b> and <b>Add Node</b> and add the Votes
- Now click on <b>Actions</b> and <b>Add Edge</b>
    - Parent node = User, Child node = Votes 
    - (if there are 2 select the 2nd / bottom most one)
- Remove all unusable fields for the benchmark, here are the ones to <b>remove</b> by node
    - Remember to <b>Save changes</b>!
    - User: Reputation, Views, DownVotes, UpVotes
    - Comments: Score
    - Posts: Score, ViewCount, AnswerCount, CommentCount, FavoriteCount
    - Votes: BountyAmount
- Now lets add the SQL on Votes and Comments to compute their label flag since Posts is already done.
    - Click on the Comments node and click <b>Edit Node</b>
    - Under <b>Labels / target variable</b> add this SQL
    ```sql
    select UserId,
    count(*) as made_comment_label
    from Comments
    group by UserId
    ```
    - save changes
    - Click on the Votes node (the one directly under the User) and click <b>Edit Node</b>
    - Under <b>Labels / target variable</b> add this SQL
    ```sql
    select UserId,
    count(*) as made_vote_label
    from Votes
    group by UserId
    ```
    - save changes

# Step 5: add final filters, as outlined in the paper, configure compute graph, and execute

- Open the User node, click <b>Edit Node</b> and add the following to SQL to <b>Annotations / special selects</b>
```sql
select *,
date_diff('day', CreationDate, date cut_date) as age_days
```
- Add the following to <b>Filters / wheres</b>
```sql
where CreationDate < date cut_date
```

# step 6: edit prefixes and add final filters
- edit Posts, Comments, Votes, and PostHistory (only the ones directly connected to Users)
- make <b>Posts</b> prefix: <b>post</b>
- make <b>Comments</b> prefix: <b>comm</b>
- make <b>Votes</b> prefix: <b>uvote</b>
- make <b>PostHistory</b> prefix: <b>ph</b>
- click on Users and add the following SQL to <b>Post-join filters</b> (very bottom)
```sql
where comm_CreationDate_max is not null
or uvote_CreationDate_max is not null
or post_CreationDate_max is not null
```
- per the relbench paper this ensures all users considered had at least some activity in history

# step 7: edit top-level compute graph
- on the nav bar click <b>Actions</b> and select <b>Show compute graph details</b>
- click <b>Edit compute graph</b> and make sure <b>automate date filters</b> is <b>checked</b>
- click <b>Save changes</b>

# step 8: execute the compute graph and get training data
- on the nav bar click <b>Actions</b> and <b>Execute compute graph</b>
- after it is done executing copy/paste the S3 url and modify the below variable to load it in

In [4]:
import pandas as pd
from torch_frame.utils import infer_df_stype
import catboost
from sklearn import metrics

In [16]:
# in the UI it will look something like:
# kurve-customers/4e1a245a-3065-4600-bb0e-a92e06ee835c/1/output/user_badge_train
# just take the part after kurve-customers/ and ovewrite the below
train_path = 'https://kurve-customers.s3.amazonaws.com/3d95ca0a-b09a-4cc7-8bbf-9c0e98932af2/3/output/user_badge_train'

In [17]:
df = pd.read_parquet(train_path)

In [18]:
df.shape

(77814, 157)

In [19]:
df.head(2)

,User_Id,User_AccountId,User_DisplayName,User_Location,User_ProfileImageUrl,User_WebsiteUrl,User_AboutMe,User_CreationDate,User_LastAccessDate,user_age_days,...,ph_num_events_60d,ph_num_events_90d,ph_num_events_180d,ph_num_events_365d,ph_num_events_730d,ph_30dv60_change,ph_60dv90_change,ph_90dv180_change,ph_180dv365_change,ph_365dv730_change
0,166286,4854026,starbuck,None,None,None,None,2017-06-22 16:04:33.980,2019-02-22 23:58:01.913,923,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,166293,11175242,jenny,None,None,None,None,2017-06-22 16:29:18.877,2017-06-23 00:55:52.137,923,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [20]:
# get the columns that flag if the user
# had a vote, post, or comment
label_cols = [c for c in df.columns if 'label' in c]
print(label_cols)

['uvote_made_vote_label', 'post_Id_label', 'comm_made_comment_label']


In [21]:
df[label_cols].head()

,uvote_made_vote_label,post_Id_label,comm_made_comment_label
0,<NA>,<NA>,<NA>
1,<NA>,<NA>,<NA>
2,<NA>,<NA>,<NA>
3,<NA>,<NA>,<NA>
4,<NA>,<NA>,<NA>


In [22]:
# now transform the label into a single target label
def had_engagement(row, label_cols):
    for col in label_cols:
        if not pd.isnull(row[col]):
            return 1
    return 0

In [23]:
target = 'had_engagement'
df[target] = df.apply(lambda row: had_engagement(row, label_cols), axis=1)

In [24]:
df[target].sum()/len(df)

0.032102192407535916

# step 9: get the test data
- on nav bar click <b>Actions</b>, <b>Show compute graph details</b>, and <b>edit compute graph</b>
- change the <b>cut date</b> to <b>01/01/2021</b>
- save changes
- execute the compute graph

In [26]:
# Kurve will OVERWRITE the file so just load the same path
# as before but in a new variable
# make sure the compute graph finishes before loading this
test = pd.read_parquet(train_path)

In [27]:
test.shape

(87789, 157)

In [28]:
# should be true!
print(test.shape[0] == 87789)

True


In [29]:
# create the target variable
# for the test set
test[target] = test.apply(lambda row: had_engagement(row, label_cols), axis=1)

In [30]:
test[target].sum()/len(test)

0.027338277005091755

In [31]:
# get the features we're going to use
stypes = infer_df_stype(df)

In [32]:
features = [
    k for k,v in stypes.items()
    if str(v) == 'numerical'
    and k not in ['User_Id', 'User_AccountId']
    and 'label' not in k and k != 'had_engagement']

In [33]:
len(features)

123

In [34]:
[f for f in features if f not in test.columns]

[]

In [35]:
# Train/eval code

import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

# -------------------------------------------------
# 1. Split off the *final* test set (once!)
# -------------------------------------------------
X_train_full, X_test, y_train_full, y_test = train_test_split(
    df[features], df[target],
    test_size=0.20,          # 20 % held-out test
    stratify=df[target],
    random_state=42
)

# -------------------------------------------------
# 2. K-Fold CV on the remaining 80 %
# -------------------------------------------------
k = 3                                   # change to 10, etc.
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

fold_aucs = []
test_preds = np.zeros(len(X_test))      # out-of-fold predictions on *test*
oof_preds = np.zeros(len(X_train_full)) # optional: OOF on training data

# CatBoost pools (optional but faster)
train_pool = Pool(X_train_full, y_train_full)  # , cat_features=cat_features)

for fold, (idx_tr, idx_va) in enumerate(skf.split(X_train_full, y_train_full), 1):
    print(f"\n=== Fold {fold} ===")
    
    X_tr, X_va = X_train_full.iloc[idx_tr], X_train_full.iloc[idx_va]
    y_tr, y_va = y_train_full.iloc[idx_tr], y_train_full.iloc[idx_va]

    # -----------------------------------------------------------------
    # 3. Fit on the *training* split of this fold
    # -----------------------------------------------------------------
    mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        use_best_model=True,
    
        # capacity vs regularization
        iterations=2000,
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[1,36],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
    
        verbose=200
    )
    mdl.fit(
        X_tr, y_tr,
        eval_set=(X_va, y_va),
        use_best_model=False,
        verbose=False
    )

    # -------------------------------------------------
    # 4. Validation AUC for this fold
    # -------------------------------------------------
    val_pred = mdl.predict_proba(X_va)[:, 1]
    val_auc  = roc_auc_score(y_va, val_pred)
    fold_aucs.append(val_auc)
    print(f"Fold {fold} validation AUC : {val_auc:.4f}")

    # -------------------------------------------------
    # 5. Accumulate predictions on the *final* test set
    # -------------------------------------------------
    test_preds += mdl.predict_proba(X_test)[:, 1] / k

    # (optional) OOF on training data
    oof_preds[idx_va] = val_pred

# -------------------------------------------------
# 6. Final metrics
# -------------------------------------------------
print("\n=== CV Summary ===")
print(f"Mean CV AUC : {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")
print(f"Folds AUC   : {[f'{a:.4f}' for a in fold_aucs]}")

test_auc = roc_auc_score(y_test, test_preds)
print(f"\nFinal test AUC (averaged over {k} folds): {test_auc:.4f}")

# -------------------------------------------------
# 7. (Optional) Refit on the *whole* train_full for deployment
# -------------------------------------------------
final_mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        #use_best_model=True,
    
        # capacity vs regularization
        #iterations=2000,
        iterations=int(mdl.best_iteration_ * 1.1),  # a bit more than best
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[1,36],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
    
        verbose=200
    )
final_mdl.fit(X_train_full, y_train_full)


=== Fold 1 ===
Fold 1 validation AUC : 0.8840

=== Fold 2 ===
Fold 2 validation AUC : 0.8810

=== Fold 3 ===
Fold 3 validation AUC : 0.8914

=== CV Summary ===
Mean CV AUC : 0.8855 ± 0.0043
Folds AUC   : ['0.8840', '0.8810', '0.8914']

Final test AUC (averaged over 3 folds): 0.8923
0:	total: 7.8ms	remaining: 2.2s
200:	total: 2.45s	remaining: 1s
282:	total: 3.49s	remaining: 0us


In [36]:
test['pred'] = final_mdl.predict_proba(test[features])[:,1]

In [37]:
print(metrics.roc_auc_score(test[target], test['pred']))

0.8970174412590224


In [40]:
# Relbench paper scores
# Data scientist 20+ hours = 0.90
# LLM: 0.81
# Kumo: 0.87
# Kumo fine tuned: 0.907

In [39]:
list(reversed(sorted(zip(final_mdl.feature_names_, final_mdl.feature_importances_), key=lambda x: x[1])))

[('ph_seconds_since_last', 9.701184556968599),
 ('post_seconds_since_last', 6.503490811712844),
 ('comm_seconds_since_last', 5.932206603745044),
 ('user_age_days', 4.897445834704313),
 ('Badg_Class_count', 4.424498452493685),
 ('post_Id_count', 3.9549128787734067),
 ('comm_ContentLicense_count', 3.279826830102761),
 ('Badg_seconds_since_last', 2.952184508805675),
 ('comm_num_events_730d', 2.938270308331868),
 ('post_ContentLicense_count', 2.69461953650855),
 ('subvote_seconds_since_last_min', 2.3950432252331684),
 ('ph_Id_count', 2.3590870824632026),
 ('Badg_Id_count', 2.2122158160473964),
 ('subvote_Id_count_sum', 1.7618224439880985),
 ('comm_365dv730_change', 1.688043838931154),
 ('subvote_seconds_since_last_avg', 1.5918210997416515),
 ('subvote_Id_count_avg', 1.5780306626113108),
 ('ph_ContentLicense_count', 1.5130850042777244),
 ('subvote_seconds_since_last_sum', 1.4614104283142075),
 ('subvote_seconds_since_last_max', 1.4476412365961728),
 ('comm_num_events_365d', 1.35579105495858